https://www.youtube.com/watch?v=rc_Y6rdBqXM

binance api key: 
https://support.coinigy.com/hc/en-us/articles/360001144614-How-do-I-find-my-API-key-on-Binance-com-

binance documentation: https://binance-docs.github.io/apidocs/spot/en/#trade-streams 

sql engine: https://docs.sqlalchemy.org/en/14/core/engines.html

In [1]:
#1.Requesting data from Binance (python)
#2.Do data cleaning and store it in a SQL database (python)
#3.Database is updated every second from the python script (python & SQL)*****
#4.Get data from the SQL database, apply a trading strategy and execute orders via binance API (python script no.2)
#5.Order excutions (Done by python scripts on Binance)

In [2]:
#1. getting live price data from binance 'Live data script'
import pandas as pd
import sqlalchemy
from binance.client import Client
from binance import BinanceSocketManager

In [3]:
api_key = 'your key'
api_secret = 'your api secret
client = Client(api_key,api_secret)

In [4]:
bsm = BinanceSocketManager(client) #provide the client

In [5]:
socket = bsm.trade_socket('BTCUSDT')

In [6]:
await socket.__aenter__()
msg = await socket.recv()
print(msg) #we only care about the price which is p
print(msg['p'])

{'e': 'trade', 'E': 1623877152817, 's': 'BTCUSDT', 't': 914100628, 'p': '38537.23000000', 'q': '0.00026200', 'b': 6484304535, 'a': 6484304306, 'T': 1623877152816, 'm': False, 'M': True}
38537.23000000


In [7]:
def createframe(msg): #transferring the msg to a dataframe
    df = pd.DataFrame([msg])
    df = df.loc[:,['s','E','p']] #we just want the symbol, event time, and price
    df.columns = ['Symbol','Time','Price']
    df.Price = df.Price.astype(float) #becuz it was string type
    df.Time = pd.to_datetime(df.Time, unit='ms') #becuz it was standard time
    return df



In [8]:
#2. Use the df and store it into a SQL database by df.to_sql() function #plz watch SQL tutorial dont skip the basic
engine = sqlalchemy.create_engine('sqlite:///BTCUSDTstream.db')
#we want to update this db everytime we receive price from binance
while True:
    await socket.__aenter__()
    msg = await socket.recv()
    frame = createframe(msg)
    frame.to_sql('BTCUSDT', engine, if_exists='append', index=False)
    print(frame)

    Symbol                    Time     Price
0  BTCUSDT 2021-06-16 20:59:12.855  38537.22
    Symbol                    Time     Price
0  BTCUSDT 2021-06-16 20:59:12.997  38537.22
    Symbol                    Time     Price
0  BTCUSDT 2021-06-16 20:59:13.220  38537.23
    Symbol                    Time    Price
0  BTCUSDT 2021-06-16 20:59:13.911  38539.2


AttributeError: 'Connect' object has no attribute 'protocol'